# NEW HOTS 01 - Homeostasis

In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd '../HOTS'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/joe/Documents/boulot/git/homhots/HOTS/HOTS


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import os
import pickle
from mix_Network import *
from Event import Event
import ipywidgets

# Test of homeostasis rule on Lagorce 2017 results

In [ ]:
homhots = network(algo='lagorce', krnlinit='first', tau=20, homeo=True)
homhots.learninglagorce(diginit=True)

  0%|          | 894/223371 [00:00<03:55, 943.99it/s] 

In [ ]:
homhots.plotlayer()
homhots.plotconv()

In [ ]:
trainmap = homhots.traininglagorce()

In [ ]:
testmap = homhots.testinglagorce(trainmap)

In [ ]:
fname = "../Records/EXP_01_LagorceKmeans/2020-12-01-mix_hots_homeo.pkl"
with open(fname, 'wb') as file:
    pickle.dump([hots, trainmap, testmap], file, pickle.HIGHEST_PROTOCOL)

# Random initialisation

In [ ]:
fname = "../Records/EXP_01_LagorceKmeans/2020-12-01-mix_hots_homeo_randominit.pkl"
if not os.path.isfile(fname):
    homhotsrdn = network(algo='lagorce', krnlinit='rdn', tau=20, homeo=True)
    homhotsrdn.learninglagorce()
    homhotsrdn.plotlayer()
    trainmaprdnhom = homhotsrdn.traininglagorce()
    testmaprdnhom = homhotsrdn.testinglagorce(trainmaprdnhom)
    with open(fname, 'wb') as file:
        pickle.dump([homhotsrdn, trainmaprdnhom, testmaprdnhom], file, pickle.HIGHEST_PROTOCOL)
else:
    with open(fname, 'rb') as file:
        homhotsrdn, trainmaprdnhom, testmaprdnhom = pickle.load(file)
    homhotsrdn.plotlayer()